# nb-model_xg-model-neural-process-1

In [15]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from functools import partial
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_xg-model-neural-process-1.ipynb'
dir_name = 'model'
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

from common_util import MODEL_DIR, is_valid, isnt, compose, pd_split_ternary_to_binary, df_del_midx_level, midx_intersect, pd_get_midx_level, pd_rows, df_midx_restack
from model.common import DATASET_DIR, XG_PROCESS_DIR, XG_DATA_DIR, XG_DIR, PYTORCH_MODELS_DIR, TEST_RATIO, VAL_RATIO, EXPECTED_NUM_HOURS, default_dataset
from model.xg_util import get_xg_feature_dfs, get_xg_label_target_dfs, get_hardcoded_daily_dfs, get_hardcoded_hourly_dfs
from model.np_util import AttentiveNP
from model.pl_np import NP
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

%autoreload 1
%aimport model.np_util, model.pl_np, model.pl_generic

Prune the xg data down to the data of interest to use in further experiments.

## Load Data

In [16]:
assets = ['sp_500', 'russell_2000', 'nasdaq_100', 'dow_jones']
chosen_asset = assets[0]

fd = get_xg_feature_dfs(chosen_asset)
ld, td = get_xg_label_target_dfs(chosen_asset)

# daily data axefiles
d_all_common = ['dwrmx', 'dwrod', 'dwrpt', 'dwrzn', 'dffd']
d_extra = ['dlogret', 'dohlca', 'ddiff', 'dc', 'dwrxmx']

# hourly data axefiles
h_all_common = ['hdmx', 'hdod', 'hdpt', 'hdzn']
h_extra = ['hdgau', 'hduni', 'hohlca']

## Select Data

In [17]:
d_pba = get_hardcoded_daily_dfs(fd, 'pba')
d_vol = get_hardcoded_daily_dfs(fd, 'vol')
d_buzz = get_hardcoded_daily_dfs(fd, 'buzz')
d_nonbuzz = get_hardcoded_daily_dfs(fd, 'buzz')
h_pba = get_hardcoded_hourly_dfs(fd, 'pba')
h_vol = get_hardcoded_hourly_dfs(fd, 'vol')
h_buzz = get_hardcoded_hourly_dfs(fd, 'buzz')

/home/kev/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Returns (AR)
chosen_f = fd['d']['pba']['dret']['pba_hlh_hdxret_dret']
chosen_f = fd['d']['pba']['dret']['pba_hoc_hdxret_dret']
#chosen_f = fd['d']['pba']['ddir']['pba_hoc_hdxret_ddir']

In [19]:
# Daily Features
# chosen_f = d_pba
chosen_f = fd['d']['pba']['ddir']['pba_hoc_hdxret_ddir']

In [20]:
# fd['d']['pba']
# fd['d']['pba']['dohlca']['pba_dohlca'].unstack().pct_change().dropna().corr()
# fd['d']['vol']['dohlca']['vol_dohlca'].unstack().pct_change().dropna().corr()

In [21]:
# Hourly Features
chosen_f = h_pba
chosen_f = fd['h']['pba']['hdmx']['pba_hohlca_hdmx']

# # Random 'Features':
# chosen_f = pd.DataFrame(np.random.rand(len(chosen_f)), index=chosen_f.index).rename({chosen_f.index.levels[1][0]:

### Choose Labels/Targets and Process

In [22]:
# predict ddir(t)
chosen_l = pd_split_ternary_to_binary(ld['hoc']['pba']['ddir'].replace(to_replace=-1, value=0))
chosen_t = pd_split_ternary_to_binary(td['hoc']['pba']['dret'])

In [23]:
# # predict thresholded direction: ddir1(t)
# chosen_l_len = len(ld['hoc']['pba']['ddir1_log'].columns)
# chosen_l = pd_split_ternary_to_binary(ld['hoc']['pba']['ddir1_log'])
# chosen_l = chosen_l.sum(level=0)#+len(chosen_l.columns)
# chosen_l = (chosen_l[chosen_l!=0] + chosen_l_len).fillna(0).astype(int)
# chosen_l = df_add_midx_level(chosen_l, 'pba_hoc_hdxret1_ddir1', loc=1, name='id1')

# chosen_t = pd_split_ternary_to_binary(td['hoc']['pba']['dret1_log']).abs().max(level=0)
# chosen_t = df_add_midx_level(chosen_t, 'pba_hoc_hdxret1_dret1', loc=1, name='id1')
# #chosen_l = chosen_l.sum(level=0)#+len(chosen_l.columns)
# #chosen_l

In [24]:
# 'Predict' the present ddir(t-1), (sanity check)
chosen_l = pd_split_ternary_to_binary(df_del_midx_level(fd['d']['pba']['ddir']['pba_hoc_hdxret_ddir'].rename(columns={-1:'pba_hoc_hdxret_ddir'}), loc=1).replace(to_replace=-1, value=0).astype(int))
chosen_t = pd_split_ternary_to_binary(df_del_midx_level(fd['d']['pba']['dret']['pba_hoc_hdxret_dret'].rename(columns={-1:'pba_hoc_hdxret_dret'}), loc=1))

### Get Common Indexed Rows (Intersect First Level of MultiIndex)

In [25]:
year_interval = ('2009', '2018')
common_idx = midx_intersect(pd_get_midx_level(chosen_f), pd_get_midx_level(chosen_l), pd_get_midx_level(chosen_t))
common_idx = common_idx[(common_idx > year_interval[0]) & (common_idx < year_interval[1])]
feature_df, label_df, target_df = map(compose(partial(pd_rows, idx=common_idx), df_midx_restack), [chosen_f, chosen_l, chosen_t])
assert(all(feature_df.index.levels[0]==label_df.index.levels[0]))
assert(all(feature_df.index.levels[0]==target_df.index.levels[0]))

## Neural Process

In [32]:
t_params = {
	'window_size': 20,
	'feat_dim': None,
	'train_shuffle': False,    
	'epochs': 50,
	'batch_size': 128,
	'batch_step_size': 64,
	'loss': 'clf',
	'opt': {
		'name': 'adam',
		'kwargs': {
			'lr': .001
		}
	},
	'sch': {
#		'name': 'rpl',
#		'kwargs': {
#			'mode': 'min',
#			'factor': 0.1,
#			'patience': 10,
#			'threshold': 0.0001,
#			'threshold_mode': 'rel',
#			'cooldown': 0,
#			'min_lr': 0
#		}
	},
	'num_workers': 0,
	'pin_memory': True
}

m_params = {
    'label_size': 1,
    'det_encoder_params': {
        'it_params': {
            
        },
        'ct_params': {
            
        },
        'tt_params': {
            
        }
    },
    'lat_encoder_params': {
    },
    'decoder_params': {
    },
    'use_lvar': False,
    'context_in_target': False
}
mdl = NP(AttentiveNP, m_params, t_params, (feature_df, label_df, target_df))

In [33]:
mdl

NP(
  (model): AttentiveNP(
    (det_encoder): DetEncoder(
      (input_encoder): StackedTCN(
        (model): Sequential(
          (rb_0): ResidualBlock(
            (net): Sequential(
              (tl_0_0): TemporalLayer1d(
                (layer): Sequential(
                  (0): Conv1d(6, 128, kernel_size=(3,), stride=(1,), padding=(2,))
                  (1): ELU(alpha=1.0)
                  (2): Dropout(p=0.0, inplace=False)
                )
              )
              (tl_0_1): TemporalLayer1d(
                (layer): Sequential(
                  (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
                  (1): ELU(alpha=1.0)
                  (2): Dropout(p=0.5, inplace=False)
                )
              )
              (tl_0_2): TemporalLayer1d(
                (layer): Sequential(
                  (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(4,))
                  (1): ELU(alpha=1.0)
        

In [34]:
escb = EarlyStopping(monitor='val_loss', min_delta=0.00, patience=30, verbose=False, mode='min')
trainer = pl.Trainer(max_epochs=t_params['epochs'], auto_lr_find=False, gpus=1, amp_level='O1', precision=16)

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
INFO:lightning:Using native 16bit precision.


In [35]:
trainer.fit(mdl)

/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)

  | Name  | Type        | Params
--------------------------------------
0 | model | AttentiveNP | 1 M   
INFO:lightning:
  | Name  | Type        | Params
--------------------------------------
0 | model | AttentiveNP | 1 M   
/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a

/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: DeprecationWarning: Method `validation_end` was deprecated in v0.7 and will be removed in v1.0. Use `validation_epoch_end` instead.
  warnings.warn(*args, **kwargs)
/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1